In [1]:
# Creates new business-level conceptual entities into the DB from an excel sheet
import sqlite3
import pandas as pd
import numpy as np
import os
from datetime import datetime
import sys
sys.path.append('../../')
from user_packages import hashing

In [2]:
# Set script variables
source_file_name = 'Org_Dictionary(Business Areas).csv'
source_file_path = '../../imports/business_area'

record_source = 'BusinessAreaCSV'

In [3]:
conn = sqlite3.connect('../../full_metadata.db')
cur = conn.cursor()

In [4]:
# read table to df
df = pd.read_csv(os.path.join(source_file_path, source_file_name))
df = df.replace({np.nan: None})

df

,BusinessModelKeyPhrase,BusinessName,BusinessAreaModelKeyPhrase,BusinessAreaName,DisplayName,Description,isDeleted,BusinessContact
0,object.one.furniture,Vivid,people.orange.all,Repairs,Repairs,Responsible for the process of either handling...,0,None
1,object.one.furniture,Vivid,people.orange.all,Scheduling,Scheduling,Making sure the right people arrive at the rig...,0,None
2,object.one.furniture,Vivid,people.orange.all,Voids,Voids,Handling maintenance/ repair/ renovation of em...,0,None
3,object.one.furniture,Vivid,people.orange.all,Lettings,Lettings,Putting tenants in properties and retrieving rent,0,None
4,object.one.furniture,Vivid,people.orange.all,Compliance,Compliance,Ensuring that regulatory requirements are met,0,None
5,object.one.furniture,Vivid,people.orange.all,AssetSustain,Asset and Sustainability,None,0,Tom Robinson
6,object.one.furniture,Vivid,people.orange.all,CustomerServices,Customer Services,None,0,Alex Nagle
7,object.one.furniture,Vivid,people.orange.all,Finance,Finance,None,0,David Ball
8,object.one.furniture,Vivid,people.orange.all,People,People,None,0,Susan Noone
9,object.one.furniture,Vivid,people.orange.all,PropertyServices,Property Services,None,0,None


In [5]:
# Create additional fields
df['RecordSource'] = record_source
df['LoadDate'] = datetime.now()

df['BusinessAreaKeyPhrase'] = df.apply(lambda row: row['BusinessName'] + '.' + row['BusinessAreaName'], axis=1)
df['ModelBusinessAreaKeyPhrase'] = df.apply(lambda row: row['BusinessAreaModelKeyPhrase'] + ':' + row['BusinessAreaKeyPhrase'], axis=1)
df['BusinessBusinessAreaKeyPhrase'] = df.apply(lambda row: row['BusinessName'] + ':' + row['BusinessAreaKeyPhrase'], axis=1)


In [13]:
# Hash fields

df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'BusinessHashKey'
  , columns = ['BusinessName']
)

df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'ModelHashKey'
  , columns = ['BusinessAreaModelKeyPhrase']
)
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'BusinessAreaHashKey'
  , columns = ['BusinessAreaKeyPhrase']
)
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'ModelBusinessAreaHashKey'
  , columns = ['ModelBusinessAreaKeyPhrase']
)

df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'BusinessBusinessAreaHashKey'
  , columns = ['BusinessBusinessAreaKeyPhrase']
)

df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'HashDiff'
  , columns = ['BusinessAreaName', 'DisplayName', 'Description', 'isDeleted', 'BusinessContact']
)

df

,BusinessModelKeyPhrase,BusinessName,BusinessAreaModelKeyPhrase,BusinessAreaName,DisplayName,Description,isDeleted,BusinessContact,RecordSource,LoadDate,BusinessAreaKeyPhrase,ModelBusinessAreaKeyPhrase,BusinessBusinessAreaKeyPhrase,BusinessHashKey,ModelHashKey,BusinessAreaHashKey,ModelBusinessAreaHashKey,HashDiff,BusinessBusinessAreaHashKey
0,object.one.furniture,Vivid,people.orange.all,Repairs,Repairs,Responsible for the process of either handling...,0,None,BusinessAreaCSV,2024-08-07 16:00:39.088097,Vivid.Repairs,people.orange.all:Vivid.Repairs,Vivid:Vivid.Repairs,ff894daac36de37f2537ff46b8d41c72,d98e8ab954365a242d5457587e96b140,0b35a2f27c61505ab4baf085e24d9e0a,23f76d2ea70b18ccd33836e2a0ed8a13,316cc0c09fa1bd28d35a3f7e9965cf23,2a12960fb8f51553812326d92b161f69
1,object.one.furniture,Vivid,people.orange.all,Scheduling,Scheduling,Making sure the right people arrive at the rig...,0,None,BusinessAreaCSV,2024-08-07 16:00:39.088097,Vivid.Scheduling,people.orange.all:Vivid.Scheduling,Vivid:Vivid.Scheduling,ff894daac36de37f2537ff46b8d41c72,d98e8ab954365a242d5457587e96b140,feb98503faf581cdb6978f5f32e14484,52da4b89adc2487f4172e91a3de385a9,ae060ac16dd043a406cc6cf63f13620b,a57d8b0aab282229b0ec6fde41b2e6b0
2,object.one.furniture,Vivid,people.orange.all,Voids,Voids,Handling maintenance/ repair/ renovation of em...,0,None,BusinessAreaCSV,2024-08-07 16:00:39.088097,Vivid.Voids,people.orange.all:Vivid.Voids,Vivid:Vivid.Voids,ff894daac36de37f2537ff46b8d41c72,d98e8ab954365a242d5457587e96b140,8a593517adf82fb71d684648da693677,ffbc0d594ff3ef82d548670ebeeaac99,931beebcb456ad6cc2ef34a47a7a09bb,954a8d33ecbe947a4a7b7cdc0aa0fcf4
3,object.one.furniture,Vivid,people.orange.all,Lettings,Lettings,Putting tenants in properties and retrieving rent,0,None,BusinessAreaCSV,2024-08-07 16:00:39.088097,Vivid.Lettings,people.orange.all:Vivid.Lettings,Vivid:Vivid.Lettings,ff894daac36de37f2537ff46b8d41c72,d98e8ab954365a242d5457587e96b140,ec614606fa800fcddf8138201dcc1fb9,03886e204c1311e8670e61245734e48c,150fac02773deae116d114400c264efd,bca344323184b194313b294329da6ba9
4,object.one.furniture,Vivid,people.orange.all,Compliance,Compliance,Ensuring that regulatory requirements are met,0,None,BusinessAreaCSV,2024-08-07 16:00:39.088097,Vivid.Compliance,people.orange.all:Vivid.Compliance,Vivid:Vivid.Compliance,ff894daac36de37f2537ff46b8d41c72,d98e8ab954365a242d5457587e96b140,dd6a3a31dc598715113616f36fef270b,0fc848a502466ec825a66de3841936bd,92bb83570e4cf1bb17e1d1dd123c6837,372319587935625cdfa0b1914ee85a1b
5,object.one.furniture,Vivid,people.orange.all,AssetSustain,Asset and Sustainability,None,0,Tom Robinson,BusinessAreaCSV,2024-08-07 16:00:39.088097,Vivid.AssetSustain,people.orange.all:Vivid.AssetSustain,Vivid:Vivid.AssetSustain,ff894daac36de37f2537ff46b8d41c72,d98e8ab954365a242d5457587e96b140,d48076c74c8587d0a4a2b380d0023fde,f3ef74535b99090b2018888bde65d3ec,e99f2c75dd2bd8bacd9d70c732007602,44a1882d3ea84a41539173cc6e75c449
6,object.one.furniture,Vivid,people.orange.all,CustomerServices,Customer Services,None,0,Alex Nagle,BusinessAreaCSV,2024-08-07 16:00:39.088097,Vivid.CustomerServices,people.orange.all:Vivid.CustomerServices,Vivid:Vivid.CustomerServices,ff894daac36de37f2537ff46b8d41c72,d98e8ab954365a242d5457587e96b140,9a8176243732c9337347415a002db5b3,12726847fe7e7197a904cca0244fd27d,b86fe1cf67b08441ec8ebe56504904fd,56b3607ac067a70ecd3168a00a689a7a
7,object.one.furniture,Vivid,people.orange.all,Finance,Finance,None,0,David Ball,BusinessAreaCSV,2024-08-07 16:00:39.088097,Vivid.Finance,people.orange.all:Vivid.Finance,Vivid:Vivid.Finance,ff894daac36de37f2537ff46b8d41c72,d98e8ab954365a242d5457587e96b140,35418af0f2db1c137ec9612a15ae40c0,1bf8536ca1f3dc536f087824fcc0da45,581dcabe0df250117e191a784fb9b273,a68943df7437bb7f61d8e644340a8f45
8,object.one.furniture,Vivid,people.orange.all,People,People,None,0,Susan Noone,BusinessAreaCSV,2024-08-07 16:00:39.088097,Vivid.People,people.orange.all:Vivid.People,Vivid:Vivid.People,ff894daac36de37f2537ff46b8d41c72,d98e8ab954365a242d5457587e96b140,

In [14]:
# Clear down and Write to staging
conn.execute("DELETE FROM stg_Py_CSVToBusinessArea")
conn.commit()
df.to_sql('stg_Py_CSVToBusinessArea', conn, if_exists='append', index=False)

11

In [9]:
# Write to the Hub (BusinessArea)
sql_query = """
INSERT INTO rv_h_BusinessArea
(
    BusinessAreaHashKey
  , LoadDate
  , RecordSource
  , BusinessAreaKeyPhrase
)
SELECT DISTINCT
    BusinessAreaHashKey
  , LoadDate
  , RecordSource
  , BusinessAreaKeyPhrase
FROM
  stg_Py_CSVToBusinessArea
WHERE
  BusinessAreaHashKey NOT IN (SELECT BusinessAreaHashKey FROM rv_h_BusinessArea)
""";
conn.execute(sql_query)
conn.commit()

In [10]:
# Write to the Satellite (BusinessArea)
sql_query = """
INSERT INTO rv_s_BusinessArea
(
    BusinessAreaHashKey
  , LoadDate
  , RecordSource
  , HashDiff
  , BusinessAreaName
  , DisplayName
  , Description
  , isDeleted
  , BusinessContact
)
SELECT DISTINCT
    stg.BusinessAreaHashKey
  , stg.LoadDate
  , stg.RecordSource
  , stg.HashDiff
  , stg.BusinessAreaName
  , stg.DisplayName
  , stg.Description
  , stg.isDeleted
  , stg.BusinessContact
FROM
  stg_Py_CSVToBusinessArea AS stg
  LEFT OUTER JOIN rv_s_BusinessArea AS sat ON (
    stg.BusinessAreaHashKey = sat.BusinessAreaHashKey
    AND sat.LoadDate = (
      SELECT MAX(z.LoadDate)
      FROM rv_s_BusinessArea AS z
      WHERE z.BusinessAreaHashKey = sat.BusinessAreaHashKey
    )
  )
WHERE
  (
    sat.HashDiff != stg.HashDiff
    OR sat.HashDiff IS NULL
  )
""";
conn.execute(sql_query)
conn.commit()

In [11]:
# Write to the Link (Model - BusinessArea)
sql_query = """
INSERT INTO rv_l_ModelBusinessArea
(
  ModelBusinessAreaHashKey
  , LoadDate
  , RecordSource
  , ModelHashKey
  , BusinessAreaHashKey
)
SELECT
  stg.ModelBusinessAreaHashKey
  , stg.LoadDate
  , stg.RecordSource
  , stg.ModelHashKey
  , stg.BusinessAreaHashKey
FROM
  stg_Py_CSVToBusinessArea stg
WHERE
  stg.ModelBusinessAreaHashKey IS NOT NULL
  AND ModelBusinessAreaHashKey NOT IN (
    SELECT ModelBusinessAreaHashKey
    FROM rv_l_ModelBusinessArea
  )
""";
conn.execute(sql_query)
conn.commit()

In [15]:
# Write to the Link (Business - BusinessArea)
sql_query = """
INSERT INTO rv_l_BusinessBusinessArea
(
  BusinessBusinessAreaHashKey
  , LoadDate
  , RecordSource
  , BusinessHashKey
  , BusinessAreaHashKey
)
SELECT
  stg.BusinessBusinessAreaHashKey
  , stg.LoadDate
  , stg.RecordSource
  , stg.BusinessHashKey
  , stg.BusinessAreaHashKey
FROM
  stg_Py_CSVToBusinessArea stg
WHERE
  stg.BusinessBusinessAreaHashKey IS NOT NULL
  AND BusinessBusinessAreaHashKey NOT IN (
    SELECT BusinessBusinessAreaHashKey
    FROM rv_l_BusinessBusinessArea
  )
""";
conn.execute(sql_query)
conn.commit()